In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
color = sns.color_palette()

from xgboost import XGBClassifier
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.datasets import load_svmlight_file

import datetime
import gc
%matplotlib inline

读入数据

In [3]:
train_data = './data/submission_3_24_0_train.txt'
dtrain = xgb.DMatrix(train_data, missing = -1)
#valid_data = './data/submission_3_24_valid.txt'
#dvalid = xgb.DMatrix(valid_data, missing = -1)
test_data = './data/submission_3_24_0_test.txt'
dtest = xgb.DMatrix(test_data, missing = -1)

In [3]:
train_data = pd.read_csv('./data/baseline_5_train.txt')
test_data = pd.read_csv('./data/baseline_5_test.txt')
y_train = train_data["is_trade"]
x_train = train_data.drop(["is_trade"],axis=1)
y_test = test_data["is_trade"]
x_test = test_data.drop(["is_trade"],axis=1)
dtrain = xgb.DMatrix(x_train, label = y_train, missing = -1)
dtest = xgb.DMatrix(x_test, label = y_test, missing = -1)

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)

In [6]:
def modelfit(alg, dtrain, useTrainCV=True, cv_folds=None, early_stopping_rounds=10):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        
        xgtrain = dtrain
        
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], folds =cv_folds,
             metrics=['auc','logloss'], early_stopping_rounds=early_stopping_rounds, verbose_eval=True)
        
        n_estimators = cvresult.shape[0]
        alg.set_params(n_estimators = n_estimators)
        
        #print(cvresult)
        #result = pd.DataFrame(cvresult)   #cv缺省返回结果为DataFrame
        #result.to_csv('my_preds.csv', index_label = 'n_estimators')
        
        # plot
        #test_means = cvresult['test-logloss-mean']
        #test_stds = cvresult['test-logloss-std'] 
        
        #train_means = cvresult['train-logloss-mean']
        #train_stds = cvresult['train-logloss-std'] 

        #x_axis = range(0, n_estimators)
        #pyplot.errorbar(x_axis, test_means, yerr=test_stds ,label='Test')
        #pyplot.errorbar(x_axis, train_means, yerr=train_stds ,label='Train')
        #pyplot.title("XGBoost n_estimators vs Log Loss")
        #pyplot.xlabel( 'n_estimators' )
        #pyplot.ylabel( 'Log Loss' )
        #pyplot.savefig( 'n_estimators_2.png' )

找初始合适值

In [ ]:
xgb1 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,  #数值大没关系，cv会自动返回合适的n_estimators
        max_depth=9,
        min_child_weight=64,
        #max_delta_step = 1,
        gamma=0,
        subsample=0.8,
        #scale_pos_weight = scale_pos_weight,
        colsample_bytree=0.8,
        colsample_bylevel=0.7,
        objective= 'binary:logistic',
        seed=3)


modelfit(xgb1,dtrain, cv_folds = kfold)

CV

In [78]:
param_test1 = {
    'max_depth':range(2,8,2),
    'min_child_weight':range(48,78,5)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(learning_rate=0.1, n_estimators=115, max_depth=2, min_child_weight=48, gamma=0, subsample=0.8,
           colsample_bytree=0.8, colsample_bylevel=0.7, objective= 'binary:logistic',seed=3), 
           param_grid = param_test1, refit = True, scoring='neg_log_loss', n_jobs=6, verbose=2, cv=5)
gsearch1.fit(x_train,y_train)

In [ ]:
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

In [ ]:
params ={'eta':0.1,
        'n_estimators':117,
        'max_depth':9,
        'min_child_weight':64,
        #'max_delta_step':1,
        'gamma':0,
        'subsample':0.8,
        'colsample_bytree':0.8,
        'colsample_bylevel':0.7,
        'objective':'binary:logistic',
        'eval_metric':'logloss',
         'alpha':0,
         'lambda':1,
        'seed':3}

#modelfit_libsvm(params, train_data, cv_folds = kfold)
#Fit the algorithm on the data
xgb1 = xgb.train(params, dtrain, num_boost_round=params['n_estimators'], early_stopping_rounds=10, evals = [(dtrain,'train'),(dtest,'valid')]) 
        
#Predict training set:
#train_predprob = xgb1.predict_proba(x_train)
#logloss = log_loss(y_train, train_predprob)

        
#Print model report:
#print("logloss of train :" )
#print(logloss)

In [ ]:
test_predprob = xgb1.predict_proba(x_test)
logloss = log_loss(y_test, test_predprob[:,1])
         
#输出模型的一些结果
print("\n关于现在这个模型 (测试集)")
print("AUC 得分: %f" % metrics.roc_auc_score(y_test, test_predprob[:,1]))
print("logloss:", logloss)

转成提交格式

In [6]:
test_predprob = xgb1.predict(dtest)
path = './data/'
test_data = pd.read_table(path+'test.txt',sep=' ')
test_data = test_data.drop_duplicates()
test_data = test_data.sort_values('context_timestamp')
test_data = test_data.reset_index()
test_data['predicted_score'] = test_predprob
test_data.set_index('index',inplace=True)
test_data.sort_index(inplace=True)
result = test_data[['instance_id','predicted_score']]
result.to_csv(path+'output_submission_26.txt',sep=' ',index = False)

In [ ]:
转成提交格式（分类版）

In [101]:
path = './data/'
sub_1 = pd.read_table(path+'submission_instance.txt',sep=' ',header = None)
sub_1['is_trade'] = test_predprob_1

In [107]:
sub_0 = pd.read_table(path+'submission_instance_0.txt',sep=' ',header = None)
sub_0['is_trade'] = test_predprob_0

In [118]:
result = pd.concat([sub_0,sub_1]).sort_values(0)['is_trade'].values

In [124]:
path = './data/'
test_data = pd.read_table(path+'test.txt',sep=' ')
test_data = test_data.drop_duplicates()
test_data = test_data.sort_values('context_timestamp')
test_data = test_data.reset_index()
test_data['predicted_score'] = result
test_data.set_index('index',inplace=True)
test_data.sort_index(inplace=True)
submission = test_data[['instance_id','predicted_score']]
submission.to_csv(path+'output_submission_gbdt.txt',sep=' ',index = False)

特征重要性

In [ ]:
fig, ax = pyplot.subplots(figsize=(36,56))
xgb.plot_importance(xgb1, max_num_features=500, height=0.8,ax=ax)
pyplot.savefig( 'featim_xgb_baseline.png' )
pyplot.show()